# Introduction

This notebook trains the LightGBM ensemble using the feature vectors generated in LightGBM_DeepFeatures_ssr.ipynb. The Optuna framework is employed to optimize hyperparameters and identify the best-performing configuration.

https://jiangliu5.github.io/imqac.github.io/

# Importing Libraries

In [ ]:
import os
import joblib
import time
import pandas as pd
import numpy as np
#import tensorflow as tf
import random
import optuna
import sklearn
import lightgbm
from pathlib import Path
from lightgbm import LGBMRegressor, early_stopping
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

# Setting Directories

In [ ]:
# Define some constants
NUM_WORKERS = 0 #os.cpu_count()
AMOUNT_TO_GET = 1.0
SEED = 42

# Define target data directory
BASELINE_NAME = f"VCIP_IMQA/VCIP"
BASELINE = Path(BASELINE_NAME)
TARGET_DIR = BASELINE / "EQ420_image"
TARGET_LABEL = BASELINE / "Labels"
TARGET_BASE = BASELINE / "IMQA"

# Setup training and test directories
TARGET_DIR.mkdir(parents=True, exist_ok=True)

# Create target model directory
MODEL_DIR = "trained_3072to128_updated"
os.makedirs(MODEL_DIR, exist_ok=True)

# Python's built-in random module
random.seed(SEED)

# NumPy
np.random.seed(SEED)

# TensorFlow
#tf.random.set_seed(SEED)

# Specifying Target Device

In [ ]:
# Check for available GPUs
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#    print(f"GPUs detected: {gpus}")
#else:
#    print("No GPU detected.")

# Extracting Features

In [ ]:
# Constant definition
N_SPLITS = 8
BATCH_SIZE = 1
train_csv = pd.read_csv(TARGET_LABEL / 'mos_fold_train.csv').sample(frac=1)
test_csv = pd.read_csv(TARGET_LABEL / 'mos_fold_test.csv').sample(frac=1)
train_vector = [1, 2, 3, 4, 5, 6, 7, 8]
test_vector = [9, 10]

train_df_orig = pd.read_csv('train_features.csv')

display(train_df_orig)

In [ ]:
# Apply PCA
n_components = 128
pca = PCA(n_components=n_components, random_state=SEED)
X_pca = pca.fit_transform(train_df_orig.drop(columns=['fold', 'mos'])) #Remove labels and fold!!, IMPORTANT!!!

# Explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()
print(f"Explained variance (128 components): {cumulative_variance[-1]:.4f}")

# Generate column names like PCA_0, PCA_1, ..., PCA_127
pca_columns = [f'PCA_{i}' for i in range(n_components)]
train_df = pd.DataFrame(X_pca, columns=pca_columns, index=train_df_orig.index)
train_df['fold'] = train_df_orig['fold']
train_df['mos'] = train_df_orig['mos']

In [ ]:
print(train_df.drop(columns=['fold', 'mos']).max().max())
print(train_df.drop(columns=['fold', 'mos']).min().min())

# Training

## First Round (2000 trials)

### Maximizing R2

In [ ]:
skf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Parameters
threshold = 0.25
z = 0.5

time_results = [] 

for study_n in range(0, 8):

    study_start = time.time()

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_vector)):

        fold_start = time.time()

        print(f"\n🔁 Study {study_n + 1} - Optimizing fold {fold + 1}/{N_SPLITS}...")

        # Prepare fold data
        fold_train = [train_vector[i] for i in train_idx]
        fold_val = train_vector[val_idx[0]]
        train_ids = train_df[train_df['fold'].isin(fold_train)]
        val_ids = train_df.loc[train_df['fold'] == fold_val]

        X_train_fold = train_ids.drop(columns=['fold', 'mos'])
        y_train_fold = train_ids['mos']
        X_val_fold = val_ids.drop(columns=['fold', 'mos'])
        y_val_fold = val_ids['mos']
        best_score = -float("inf")

        def fold_objective(trial):

            global best_score
            
            # first round
            lgb_model = LGBMRegressor(
                random_state=SEED + study_n * 10 + fold,
                verbosity=-1,
                objective='regression',
                boosting_type='gbdt',

                # Less trees, more regularization
                n_estimators=trial.suggest_int('n_estimators', 50, 200),
                learning_rate=trial.suggest_float('learning_rate', 0.001, 0.5, log=True),

                # Regularization
                reg_alpha=trial.suggest_float('reg_alpha', 1e-4, 1.0, log=True),
                reg_lambda=trial.suggest_float('reg_lambda', 1e-4, 1.0, log=True),

                # Conservative model complexity
                max_depth=trial.suggest_int('max_depth', 3, 8),
                num_leaves=trial.suggest_int('num_leaves', 4, 24),

                # Row and feature sampling
                colsample_bytree=trial.suggest_float('colsample_bytree', 0.7, 1.0),
                subsample=trial.suggest_float('subsample', 0.6, 0.9),
                subsample_freq=1,

                # Leaf-wise control
                min_child_samples=trial.suggest_int('min_child_samples', 10, 60),

                # Optional new tuning dimensions:
                max_bin=trial.suggest_categorical('max_bin', [255, 512, 768]),

                #device='gpu'
            )            
            
            lgb_model.fit(
                X_train_fold,
                y_train_fold,
                #sample_weight=smooth_weights(y_train_fold.values),
                eval_set=[(X_val_fold, y_val_fold)],
                eval_metric='rmse',
                callbacks=[early_stopping(stopping_rounds=30,  verbose=False)])
            preds = lgb_model.predict(X_val_fold)
            score = r2_score(y_val_fold, preds)

            # Save best model
            if score > best_score:
                best_score = score
                joblib.dump(lgb_model, f"{MODEL_DIR}/profiling_lgbm_model_fold_r2_{fold+1}_{study_n+1}_1.pkl")
                
            return score

        # Run optimization for the current fold
        study = optuna.create_study(direction='maximize')
        study.optimize(fold_objective, n_trials=1000, n_jobs=1)
        best_params = study.best_trial.params
        best_model = joblib.load(f"{MODEL_DIR}/profiling_lgbm_model_fold_r2_{fold+1}_{study_n+1}_1.pkl")  
        preds_best = best_model.predict(X_val_fold)

        print(f"✅ Best R²: {r2_score(y_val_fold, preds_best):.4f}")
        print(f"✅ Fold {fold + 1} best R²: {study.best_value:.4f}")
        print(f"🛠️  Best hyperparameters: {best_params}")

        fold_end = time.time()
        fold_time = (fold_end - fold_start) / 60

        # save into results
        time_results.append({
            "Study": study_n + 1,
            "Fold": fold + 1,
            "Training Time (min)": fold_time
        })

        print(f"⏱️ Fold {fold + 1} training time: {fold_time:.2f} min")
    
    study_end = time.time()
    study_time = (study_end - study_start) / 60
    print(f"📊 Study {study_n + 1} total training time: {study_time:.2f} min")

# Convert time_results to dataframe
df_times = pd.DataFrame(time_results)

# Show results
print(df_times)

# Average per study
avg_per_study = df_times.groupby("Study")["Training Time (min)"].mean().reset_index()
print(avg_per_study)

### Minimizing MSE

In [ ]:
skf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Parameters
threshold = 0.25
z = 0.5

for study_n in range(0, 8):

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_vector)):
        print(f"\n🔁 Study {study_n + 1} - Optimizing fold {fold + 1}/{N_SPLITS}...")

        # Prepare fold data
        fold_train = [train_vector[i] for i in train_idx]
        fold_val = train_vector[val_idx[0]]
        train_ids = train_df[train_df['fold'].isin(fold_train)]
        val_ids = train_df.loc[train_df['fold'] == fold_val]

        X_train_fold = train_ids.drop(columns=['fold', 'mos'])
        y_train_fold = train_ids['mos']
        X_val_fold = val_ids.drop(columns=['fold', 'mos'])
        y_val_fold = val_ids['mos']
        best_score = float("inf")

        def fold_objective(trial):

            global best_score
            
            # first round
            lgb_model = LGBMRegressor(
                random_state=SEED + study_n * 10 + fold,
                verbosity=-1,
                objective='regression',
                boosting_type='gbdt',

                # Less trees, more regularization
                n_estimators=trial.suggest_int('n_estimators', 50, 200),
                learning_rate=trial.suggest_float('learning_rate', 0.001, 0.5, log=True),

                # Regularization
                reg_alpha=trial.suggest_float('reg_alpha', 1e-4, 1.0, log=True),
                reg_lambda=trial.suggest_float('reg_lambda', 1e-4, 1.0, log=True),

                # Conservative model complexity
                max_depth=trial.suggest_int('max_depth', 3, 8),
                num_leaves=trial.suggest_int('num_leaves', 4, 24),

                # Row and feature sampling
                colsample_bytree=trial.suggest_float('colsample_bytree', 0.7, 1.0),
                subsample=trial.suggest_float('subsample', 0.6, 0.9),
                subsample_freq=1,

                # Leaf-wise control
                min_child_samples=trial.suggest_int('min_child_samples', 10, 60),

                # Optional new tuning dimensions:
                max_bin=trial.suggest_categorical('max_bin', [255, 512, 768]),

            )            
            
            lgb_model.fit(
                X_train_fold,
                y_train_fold,
                #sample_weight=smooth_weights(y_train_fold.values),
                eval_set=[(X_val_fold, y_val_fold)],
                eval_metric='rmse',
                callbacks=[early_stopping(stopping_rounds=30,  verbose=False)])
            preds = lgb_model.predict(X_val_fold)
            score = mean_squared_error(y_val_fold, preds)

            # Save best model
            if score < best_score:
                best_score = score
                joblib.dump(lgb_model, f"{MODEL_DIR}/lgbm_model_fold_mse_{fold+1}_{study_n+1}_1.pkl")
                
            return score

        # Run optimization for the current fold
        study = optuna.create_study(direction='minimize')
        study.optimize(fold_objective, n_trials=2000, n_jobs=1)
        best_params = study.best_trial.params
        best_model = joblib.load(f"{MODEL_DIR}/lgbm_model_fold_mse_{fold+1}_{study_n+1}_1.pkl")  
        preds_best = best_model.predict(X_val_fold)

        print(f"✅ Best mse: {mean_squared_error(y_val_fold, preds_best):.4f}")
        print(f"✅ Fold {fold + 1} best mse: {study.best_value:.4f}")
        print(f"🛠️  Best hyperparameters: {best_params}")

## Second Round (1000 trials)

### Maximizing R2

In [ ]:
skf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Parameters
threshold = 0.25
z = 0.5

for study_n in range(0, 8):

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_vector)):
        print(f"\n🔁 Study {study_n + 1} - Optimizing fold {fold + 1}/{N_SPLITS}...")

        # Prepare fold data
        fold_train = [train_vector[i] for i in train_idx]
        fold_val = train_vector[val_idx[0]]
        train_ids = train_df[train_df['fold'].isin(fold_train)]
        val_ids = train_df.loc[train_df['fold'] == fold_val]

        X_train_fold = train_ids.drop(columns=['fold', 'mos'])
        y_train_fold = train_ids['mos']
        X_val_fold = val_ids.drop(columns=['fold', 'mos'])
        y_val_fold = val_ids['mos']
        best_score = -float("inf")

        def fold_objective(trial):

            global best_score
            
            lgb_model = LGBMRegressor(
                random_state=SEED + study_n * 10 + fold,
                verbosity=-1,
                objective='regression',
                boosting_type='gbdt',

                # Narrowed search for faster convergence
                n_estimators=trial.suggest_int('n_estimators', 100, 200),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 1, log=True),

                # Focused regularization
                reg_alpha=trial.suggest_float('reg_alpha', 1e-3, 0.5, log=True),
                reg_lambda=trial.suggest_float('reg_lambda', 1e-3, 0.5, log=True),

                # Tight complexity control
                max_depth=trial.suggest_int('max_depth', 3, 7),
                num_leaves=trial.suggest_int('num_leaves', 6, 24),

                # Sampling (still some flexibility)
                colsample_bytree=trial.suggest_float('colsample_bytree', 0.75, 1.0),
                subsample=trial.suggest_float('subsample', 0.7, 0.9),
                subsample_freq=1,

                # Leaf-wise control (slightly narrower)
                min_child_samples=trial.suggest_int('min_child_samples', 20, 50),

                # Optional new tuning dimensions:
                max_bin=trial.suggest_categorical('max_bin', [255, 512, 768]),
                #min_split_gain=trial.suggest_float('min_split_gain', 0.0, 0.1),
            )            
            
            lgb_model.fit(
                X_train_fold,
                y_train_fold,
                #sample_weight=smooth_weights(y_train_fold.values),
                eval_set=[(X_val_fold, y_val_fold)],
                eval_metric='rmse',
                callbacks=[early_stopping(stopping_rounds=30,  verbose=False)])
            preds = lgb_model.predict(X_val_fold)
            score = r2_score(y_val_fold, preds)

            # Save best model
            if score > best_score:
                best_score = score
                joblib.dump(lgb_model, f"{MODEL_DIR}/lgbm_model_fold_r2_{fold+1}_{study_n+1}_2.pkl")
                
            return score

        # Run optimization for the current fold
        study = optuna.create_study(direction='maximize')
        study.optimize(fold_objective, n_trials=1000, n_jobs=1)
        best_params = study.best_trial.params
        best_model = joblib.load(f"{MODEL_DIR}/lgbm_model_fold_r2_{fold+1}_{study_n+1}_2.pkl")  
        preds_best = best_model.predict(X_val_fold)

        print(f"✅ Best R²: {r2_score(y_val_fold, preds_best):.4f}")
        print(f"✅ Fold {fold + 1} best R²: {study.best_value:.4f}")
        print(f"🛠️  Best hyperparameters: {best_params}")

### Minimizing MSE

In [ ]:
skf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Parameters
threshold = 0.25
z = 0.5

for study_n in range(0, 8):

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_vector)):
        print(f"\n🔁 Study {study_n + 1} - Optimizing fold {fold + 1}/{N_SPLITS}...")

        # Prepare fold data
        fold_train = [train_vector[i] for i in train_idx]
        fold_val = train_vector[val_idx[0]]
        train_ids = train_df[train_df['fold'].isin(fold_train)]
        val_ids = train_df.loc[train_df['fold'] == fold_val]

        X_train_fold = train_ids.drop(columns=['fold', 'mos'])
        y_train_fold = train_ids['mos']
        X_val_fold = val_ids.drop(columns=['fold', 'mos'])
        y_val_fold = val_ids['mos']
        best_score = float("inf")

        def fold_objective(trial):

            global best_score
            
            lgb_model = LGBMRegressor(
                random_state=SEED + study_n * 10 + fold,
                verbosity=-1,
                objective='regression',
                boosting_type='gbdt',

                # Narrowed search for faster convergence
                n_estimators=trial.suggest_int('n_estimators', 100, 200),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 1, log=True),

                # Focused regularization
                reg_alpha=trial.suggest_float('reg_alpha', 1e-3, 0.5, log=True),
                reg_lambda=trial.suggest_float('reg_lambda', 1e-3, 0.5, log=True),

                # Tight complexity control
                max_depth=trial.suggest_int('max_depth', 3, 7),
                num_leaves=trial.suggest_int('num_leaves', 6, 24),

                # Sampling (still some flexibility)
                colsample_bytree=trial.suggest_float('colsample_bytree', 0.75, 1.0),
                subsample=trial.suggest_float('subsample', 0.7, 0.9),
                subsample_freq=1,

                # Leaf-wise control (slightly narrower)
                min_child_samples=trial.suggest_int('min_child_samples', 20, 50),

                # Optional new tuning dimensions:
                max_bin=trial.suggest_categorical('max_bin', [255, 512, 768]),
                #min_split_gain=trial.suggest_float('min_split_gain', 0.0, 0.1),
            )            
            
            lgb_model.fit(
                X_train_fold,
                y_train_fold,
                #sample_weight=smooth_weights(y_train_fold.values),
                eval_set=[(X_val_fold, y_val_fold)],
                eval_metric='rmse',
                callbacks=[early_stopping(stopping_rounds=30,  verbose=False)])
            preds = lgb_model.predict(X_val_fold)
            score = mean_squared_error(y_val_fold, preds)

            # Save best model
            if score < best_score:
                best_score = score
                joblib.dump(lgb_model, f"{MODEL_DIR}/lgbm_model_fold_mse_{fold+1}_{study_n+1}_2.pkl")
                
            return score

        # Run optimization for the current fold
        study = optuna.create_study(direction='minimize')
        study.optimize(fold_objective, n_trials=1000, n_jobs=1)
        best_params = study.best_trial.params
        best_model = joblib.load(f"{MODEL_DIR}/lgbm_model_fold_mse_{fold+1}_{study_n+1}_2.pkl")  
        preds_best = best_model.predict(X_val_fold)

        print(f"✅ Best mse: {mean_squared_error(y_val_fold, preds_best):.4f}")
        print(f"✅ Fold {fold + 1} best mse: {study.best_value:.4f}")
        print(f"🛠️  Best hyperparameters: {best_params}")

# Third Round (1000 trials)

### Maximizing R2

In [ ]:
skf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Parameters
threshold = 0.25
z = 0.5

for study_n in range(0, 8):

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_vector)):
        print(f"\n🔁 Study {study_n + 1} - Optimizing fold {fold + 1}/{N_SPLITS}...")

        # Prepare fold data
        fold_train = [train_vector[i] for i in train_idx]
        fold_val = train_vector[val_idx[0]]
        train_ids = train_df[train_df['fold'].isin(fold_train)]
        val_ids = train_df.loc[train_df['fold'] == fold_val]

        X_train_fold = train_ids.drop(columns=['fold', 'mos'])
        y_train_fold = train_ids['mos']
        X_val_fold = val_ids.drop(columns=['fold', 'mos'])
        y_val_fold = val_ids['mos']
        best_score = -float("inf")

        def fold_objective(trial):

            global best_score
            
            lgb_model = LGBMRegressor(
                random_state=SEED + study_n * 10 + fold,
                verbosity=-1,
                objective='regression',
                boosting_type='gbdt',

                # Narrowed search for faster convergence
                n_estimators=trial.suggest_int('n_estimators', 100, 200),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 1, log=True),

                # Focused regularization
                reg_alpha=trial.suggest_float('reg_alpha', 1e-3, 0.5, log=True),
                reg_lambda=trial.suggest_float('reg_lambda', 1e-3, 0.5, log=True),

                # Tight complexity control
                max_depth=trial.suggest_int('max_depth', 3, 7),
                num_leaves=trial.suggest_int('num_leaves', 6, 24),

                # Sampling (still some flexibility)
                colsample_bytree=trial.suggest_float('colsample_bytree', 0.75, 1.0),
                subsample=trial.suggest_float('subsample', 0.7, 0.9),
                subsample_freq=1,

                # Leaf-wise control (slightly narrower)
                min_child_samples=trial.suggest_int('min_child_samples', 20, 50),

                # Optional new tuning dimensions:
                max_bin=trial.suggest_categorical('max_bin', [255, 512, 768]),
                min_split_gain=trial.suggest_float('min_split_gain', 0.0, 0.1),
            )            
            
            lgb_model.fit(
                X_train_fold,
                y_train_fold,
                #sample_weight=smooth_weights(y_train_fold.values),
                eval_set=[(X_val_fold, y_val_fold)],
                eval_metric='rmse',
                callbacks=[early_stopping(stopping_rounds=30,  verbose=False)])
            preds = lgb_model.predict(X_val_fold)
            score = r2_score(y_val_fold, preds)

            # Save best model
            if score > best_score:
                best_score = score
                joblib.dump(lgb_model, f"{MODEL_DIR}/lgbm_model_fold_r2_{fold+1}_{study_n+1}_3.pkl")
                
            return score

        # Run optimization for the current fold
        study = optuna.create_study(direction='maximize')
        study.optimize(fold_objective, n_trials=1000, n_jobs=1)
        best_params = study.best_trial.params
        best_model = joblib.load(f"{MODEL_DIR}/lgbm_model_fold_r2_{fold+1}_{study_n+1}_3.pkl")  
        preds_best = best_model.predict(X_val_fold)

        print(f"✅ Best R²: {r2_score(y_val_fold, preds_best):.4f}")
        print(f"✅ Fold {fold + 1} best R²: {study.best_value:.4f}")
        print(f"🛠️  Best hyperparameters: {best_params}")

### Minimizing MSE

In [ ]:
skf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Parameters
threshold = 0.25
z = 0.5

for study_n in range(0, 8):

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_vector)):
        print(f"\n🔁 Study {study_n + 1} - Optimizing fold {fold + 1}/{N_SPLITS}...")

        # Prepare fold data
        fold_train = [train_vector[i] for i in train_idx]
        fold_val = train_vector[val_idx[0]]
        train_ids = train_df[train_df['fold'].isin(fold_train)]
        val_ids = train_df.loc[train_df['fold'] == fold_val]

        X_train_fold = train_ids.drop(columns=['fold', 'mos'])
        y_train_fold = train_ids['mos']
        X_val_fold = val_ids.drop(columns=['fold', 'mos'])
        y_val_fold = val_ids['mos']
        best_score = float("inf")

        def fold_objective(trial):

            global best_score
            
            lgb_model = LGBMRegressor(
                random_state=SEED + study_n * 10 + fold,
                verbosity=-1,
                objective='regression',
                boosting_type='gbdt',

                # Narrowed search for faster convergence
                n_estimators=trial.suggest_int('n_estimators', 100, 200),
                learning_rate=trial.suggest_float('learning_rate', 0.01, 1, log=True),

                # Focused regularization
                reg_alpha=trial.suggest_float('reg_alpha', 1e-3, 0.5, log=True),
                reg_lambda=trial.suggest_float('reg_lambda', 1e-3, 0.5, log=True),

                # Tight complexity control
                max_depth=trial.suggest_int('max_depth', 3, 7),
                num_leaves=trial.suggest_int('num_leaves', 6, 24),

                # Sampling (still some flexibility)
                colsample_bytree=trial.suggest_float('colsample_bytree', 0.75, 1.0),
                subsample=trial.suggest_float('subsample', 0.7, 0.9),
                subsample_freq=1,

                # Leaf-wise control (slightly narrower)
                min_child_samples=trial.suggest_int('min_child_samples', 20, 50),

                # Optional new tuning dimensions:
                max_bin=trial.suggest_categorical('max_bin', [255, 512, 768]),
                min_split_gain=trial.suggest_float('min_split_gain', 0.0, 0.1),
            )            
            
            lgb_model.fit(
                X_train_fold,
                y_train_fold,
                #sample_weight=smooth_weights(y_train_fold.values),
                eval_set=[(X_val_fold, y_val_fold)],
                eval_metric='rmse',
                callbacks=[early_stopping(stopping_rounds=30,  verbose=False)])
            preds = lgb_model.predict(X_val_fold)
            score = mean_squared_error(y_val_fold, preds)

            # Save best model
            if score < best_score:
                best_score = score
                joblib.dump(lgb_model, f"{MODEL_DIR}/lgbm_model_fold_mse_{fold+1}_{study_n+1}_3.pkl")
                
            return score

        # Run optimization for the current fold
        study = optuna.create_study(direction='minimize')
        study.optimize(fold_objective, n_trials=1000, n_jobs=1)
        best_params = study.best_trial.params
        best_model = joblib.load(f"{MODEL_DIR}/lgbm_model_fold_mse_{fold+1}_{study_n+1}_3.pkl")  
        preds_best = best_model.predict(X_val_fold)

        print(f"✅ Best mse: {mean_squared_error(y_val_fold, preds_best):.4f}")
        print(f"✅ Fold {fold + 1} best mse: {study.best_value:.4f}")
        print(f"🛠️  Best hyperparameters: {best_params}")